# Initial Results of Training LLaMA 3.1 8B

## Set-up

### Python

Python is required to run this project. Although any version `> 3.11` should suffice, its's recommended to use `12.3`.
For information on installing Python for your system, take a look at the [official instructions](https://www.python.org/downloads/).

In [1]:
import sys
print(sys.version)

3.12.6 (main, Sep 14 2024, 15:48:12) [GCC 11.4.0]


### CUDA

There are a few different backends available for training LLMs depending on your available hardware. Some examples are CUDA (for Nvidia GPUs) or MPS (for Apple Silicon). These different backands may have varying success for different model sizes. Since this project intends to use relatively large models, it's recommended to use CUDA.

To install the CUDA backend visit the [offical instructions](https://developer.nvidia.com/cuda-downloads).

### PyTorch

Installing PyTorch depends on what backend you intend to use it with. For more information visit the [offical instructions](https://pytorch.org/get-started/locally/).

In [2]:
import torch

x = torch.rand(5, 3)
print(x)

torch_version = torch.__version__
torch.cuda.is_available()

tensor([[0.9460, 0.4784, 0.2143],
        [0.5331, 0.3533, 0.5600],
        [0.2525, 0.6233, 0.5755],
        [0.5302, 0.4456, 0.0093],
        [0.1918, 0.2948, 0.3110]])


True

### Hugging Face

Hugging Face is a popular AI and machine learning platform known for its open-source tools and libraries designed to make the development, deployment, and sharing of natural language processing (NLP) models easier. It provides repositories for pre-trained models. Hugging Face’s libraries, such as transformers, datasets, and tokenizers, offer simple APIs for integrating state-of-the-art models into applications. The platform also supports fine-tuning models and training them for custom use cases, while its model hub allows users to collaborate by uploading and sharing models.

### Unsloth

Unsloth is a lightweight framework designed to optimize the training and fine-tuning of large language models, such as LLaMA, with a focus on memory efficiency and reduced resource consumption. It supports advanced techniques like LoRA (Low-Rank Adaptation) and 4-bit quantization, allowing users to train models with lower VRAM requirements. By integrating gradient checkpointing and memory-efficient features, Unsloth enables developers to fit larger models and handle longer context sequences.

In [3]:
# We use unsloth to fit LLaMA-3.1-8B on our system
from unsloth import FastLanguageModel

# We use the 4-bit quantization model from unsloth
pre_trained_model = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection
load_in_4bit = True # Use 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = pre_trained_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# These parameters are recommended by unsloth for minimum VRAM.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

/home/wrighted/repos/uvic/pengullama/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060 SUPER. Max memory: 8.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Alpaca prompt

An Alpaca prompt refers to the style of prompt used in Stanford's Alpaca model, which was fine-tuned on a dataset derived from OpenAI's GPT-3. The prompts typically follow a structured format where a task instruction is paired with an optional input and expected response. This format is designed to improve the model's performance in following instructions.

The typical Alpaca prompt format looks like this:

In [4]:
alpaca_prompt = """

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return { "text" : texts, }
pass

In [5]:
from datasets import load_dataset
dataset = load_dataset('json', data_files={'train': 'json_example.json'}, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

### Training

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.392300
2,0.392300
3,0.378200
4,0.327100
5,0.267100
6,0.197800
7,0.122800
8,0.075900
9,0.037900
10,0.024100


### Time to test our model!

In [7]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Update the function to handle authentication.",
        "```python\ndef login(user):\n    return user.is_not_authenticated\n```",
        ""
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>

### Instruction:
Update the function to handle authentication.

### Input:
```python
def login(user):
    return user.is_not_authenticated
```

### Response:
```diff
--- a/app/login.py
+++ b/app/login.py
@@ -1,4 +1,4 @@
-def login(user):
-    return user.is_not_authenticated
+def login(user):
+    if user.is_authenticated:
+        return True
+    return False
```<|end_of_text|>


In [8]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "How many states are in the USA?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>

### Instruction:
How many states are in the USA?

### Input:


### Response:
```diff
--- a/app/login.py
+++ b/app/login.py
@@ -1,4 +1,4 @@
-def login(user):
-    return user.is_authenticated
+def login(user):
+    return user.is_authenticated
```<|end_of_text|>


### Results

We can see that out custom trained model has definitely changed the outputs of LLaMA. For better or for worse.